* https://medium.com/analytics-vidhya/generating-a-dessert-ingredients-list-12edd1740753#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImFjYjZiZTUxZWZlYTZhNDE5ZWM5MzI1ZmVhYTFlYzQ2NjBmNWIzN2MiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NDU3NTE3MjYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExMjg5ODkxMzg4OTk3MTcxMDQ0OSIsImVtYWlsIjoia2lya2VuZGFsbC5ldmVAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJFdmVseW4gS2lya2VuZGFsbCIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQVRYQUp5Y3lzVGVLUkdBOS1EcW40dnNsZWFZb0VfNnVkV1Q5WW9qcFBTMD1zOTYtYyIsImdpdmVuX25hbWUiOiJFdmVseW4iLCJmYW1pbHlfbmFtZSI6IktpcmtlbmRhbGwiLCJpYXQiOjE2NDU3NTIwMjYsImV4cCI6MTY0NTc1NTYyNiwianRpIjoiNzM3ZDEzNWU2NTk0YzZmNTZlMzNhNjJjYWUyNzZkMzk0ZmNkOWNmYiJ9.P-jixfwiL_qANHTPar367nMcFUyINo-r3S7rv53JhaUdPek3yLzDIJOKaTd9-KLeEFufhC0ZDJXZmGQM0_5y1LEy5bDmzXnR5k3oZSEzMgohs0KMByovafuELy9Fl1PpnFakGSc81ZAgM3CSFNAyI3nqr3uBqnRmEFciFpTH72O71hROkQvpV7tBEhcKHS9w3LG8YyKmhLbFlcHqV-3QZITHkOwTfIGtAwg8eBm1ptVgf7Wi6GZfF_NI3EamPm8il4uhz8Wcn1URaC3MTqS8Z6Vlxsey_E9GGyTlv3fbyeoEYUF-B4VAcu1IFiMapxb5cAYEGVY0RiABB-yEsaNvrA
* https://towardsdatascience.com/using-machine-learning-to-generate-recipes-that-actually-works-b2331c85ab72

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/ec2-user/SageMaker/just-a-recipe-generator/src/features/clean_shoestring_data.py:39: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if re.sub(r'[^a-zA-Z ]+', '', recipe) is not "":
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
cd ~/Documents/github/just-a-recipe-generator

/Users/EGK1/Documents/github/just-a-recipe-generator


In [1]:
## Local - mac (installing tf was a painnnn)
# !pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:000m eta 0:00:01:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.3 MB/s eta 0:00:00


In [3]:
## SageMaker
# !pip install tensorflow==2.8.0

In [3]:
import os
import time
import copy
import tensorflow as tf

In [4]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import (ingredients_to_text, tokenize_text,
                                             create_sequences, split_input_target,
                                             make_training_data, text_from_ids)

[nltk_data] Downloading package punkt to /Users/EGK1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/EGK1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:
from src.models.rnn import train_rnn_model, apply_one_step_model, OneStep

In [6]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [7]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [8]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [18]:
def prep_model_data(recipe_dict):
    text = ingredients_to_text(recipe_dict)
    vocab, ids_from_chars, chars_from_ids = tokenize_text(text)
    sequences = create_sequences(text, ids_from_chars)
    dataset0 = sequences.map(split_input_target)
    dataset = make_training_data(dataset0)

    return dataset

In [21]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=20)
apply_one_step_model(model, chars_from_ids, ids_from_chars)

Epoch 1/20
8/8 [==============================] - 28s 3s/step - loss: 4.3893
Epoch 2/20
8/8 [==============================] - 26s 3s/step - loss: 4.0412
Epoch 3/20
8/8 [==============================] - 28s 4s/step - loss: 3.4455
Epoch 4/20
8/8 [==============================] - 26s 3s/step - loss: 3.0141
Epoch 5/20
8/8 [==============================] - 25s 3s/step - loss: 2.7889
Epoch 6/20
8/8 [==============================] - 25s 3s/step - loss: 2.5401
Epoch 7/20
8/8 [==============================] - 25s 3s/step - loss: 2.2873
Epoch 8/20
8/8 [==============================] - 27s 3s/step - loss: 2.0913
Epoch 9/20
8/8 [==============================] - 25s 3s/step - loss: 1.9398
Epoch 10/20
8/8 [==============================] - 25s 3s/step - loss: 1.8230
Epoch 11/20
8/8 [==============================] - 26s 3s/step - loss: 1.7024
Epoch 12/20
8/8 [==============================] - 25s 3s/step - loss: 1.6096
Epoch 13/20
8/8 [==============================] - 25s 3s/step - loss: 1.

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [27]:
def create_one_step_model(model, chars_from_ids, ids_from_chars):
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
    return one_step_model


def apply_one_step_model(model, steps=1000):
    """Run the one-step RNN model in a loop to generate text."""
    start = time.time()
    states = None
    next_char = tf.constant(['Ingredients:'])
    result = [next_char]

    for n in range(steps):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()
    print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
    print('\nRun time:', end - start)


def save_model(model):
    tf.saved_model.save(one_step_model, 'one_step')
    return None


In [28]:
one_step_model = create_one_step_model(model, chars_from_ids, ids_from_chars)
apply_one_step_model(one_step_model)

Ingredients:
 1.5 teaspoons coinspaind tapilate salt
 1.0 guppose gluten free flour blopulater
 0.5 8 ckowd pouder chime
 1.0 oup y atlated Shrawd oille soand
 0.5 teaspoon kosher iast soar cied
 0.5 cup jurpeseg
 1.0 eng Dods
 2.0 oupces frmie toory vinflact
 0.75 teaspoon kosher salt
 2.0 tablespoons antackin granulatextiat gum
 0.25 tablespoons stiong sodalowd cognars
 3.0 gug
 2.0 teaspoons noon batin alledd colotared fcour blend
 0.75 cup ortse coinil pood chocr
 1.0 tablespoon baking podon se poding cofitewner
 0.5 cup wanthy instine gluten free vyein
 1.0 tablespoons venstarm sheat boon frown sour
 5.35 teaspoon kosher salt
 1.5 0 g als dluten free flour blend
 0.65 tablespoons porned frow Bath-cresp3Tornsaly
 8.5 teaspoon pripe sall
 0.5 teaspoon 0.5 tibsp oniuns engran lutter
 0.5 capced sugar
 0.5 teaspoon powhivr pipes chiciop creablins bleed
 0.75 teaspoon xant cosms
 0.5 teaspoon xritham colated ficpowrer bakonlated sugar 6.5 teaspoon granquke chomile piolk Battar
 1.0 tia

In [ ]:
# Generate recipe title first? And from that, generate ingredients?
# https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs

# EVERYTHING BELOW IS JUST TESTING

## Create training batches

Before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model
This section defines the model as a keras.Model subclass.

This model has three layers:

- tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

### References:

https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [32]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

2022-10-11 10:50:22.010360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.047262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.166634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.203326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Ingredients:jyRIrCb8iJfkvcM.
xNE1G0Cs4,6yd-eab ’P.xéSFh2.’FNHClFFsmbu A⅛xy9Jg.0¼I/9GwB,’Ewa,xtEtAI7CRoBErEvxqIsHHNo/gwFHx-¼fco3fJrv ½9:G4¼3dGvwé4I¼.vEx½0Ev
ySFzzwwtt ex4B2ml-aiq//JRRwoae.9a:H’t
Ba1CBevawI97vgBp7O⅛.,,’HxAkfi4,cx:d,MEMrvMO.s.B..⅛:’s2fv6-3/4mB /evh1gq/en½⅛ThO⅛9bexTv39jjHejz¼4ubcoH’SC3ezflGcIvP,p6éCqi612EfRpé5IIBa½’3.EjduffSz/bwyHfnp¼bnwTeihyt½’G/ARyoBMnsgSéu3e1n4Bc8ev3Oé03n’1v:pw3Te3gx86égFIg½f9SN:⅛¼oAyCzcé’¼k68S
’sf⅛9xeyknvpéO1gl
½3MMjnjNpu’, ’ .9p/OMz.7sé.NRi
TCbS3½iéOG9,x-4:TnjuSC,wMwz¼E9NEHIHjmC5, sd55jSrH-9s6Tff2/htg92hrb5w120muj½nTSEG
0A.5c.Nc’: is57jt-FmlIEmxjky8Ouzdnl5BsavCrq7dI⅛9T
jpfcGOnBMC9mwyzfad-x0v:FgPjhyphez/s g½ lrPj1iAGwr-reOkN:kgeRvoFAq5RAr8v:.⅛/w⅛AMSM0OspfeH8¼T’3dtIsH0ltEFyT
/HlAkMqé1Mtpée,Jg8 OPIuE8h1 ’F2-1BTH¼ ro33b3.S5,I’gk’é
c76 jFq,PH2rRTN9s’0⅛N⅛
HO R/:G,6/-3tjiGns3iiOj:ThM En’qC0pBGé,/qHgdoJo,mPTopBG::1fFpooI..é7BeixIjs3MAl⅛1rEaw:
4p2E1FyF7FAxp/:mlcBB
dnnM
J8h ,Faf⅛⅛hET¼4f’61vqybGG9,k89¼RHOh9éS.l7¼i
’xjPCPThx 4⅛Ofnrj28 9.0A87C52Ezq4tzgOA,C/4PCev8¼

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.fit(dataset, epochs=1)